In [32]:
# ===========================================
# 🧱 CELL 1 — Import Libraries and Setup
# ===========================================

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Display settings for clarity
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

# Define data paths
data_path = '../data/home-credit-default-risk/'   # adjust if needed
save_path = data_path

print("Libraries imported and paths set ✅")


Libraries imported and paths set ✅


In [33]:
# ===========================================
# 📊 CELL 2 — Load the Datasets
# ===========================================

app = pd.read_csv(data_path + 'application_train.csv')
bureau = pd.read_csv(data_path + 'bureau.csv')
prev = pd.read_csv(data_path + 'previous_application.csv')

print("✅ Datasets loaded successfully:")
print(f"Application: {app.shape}")
print(f"Bureau: {bureau.shape}")
print(f"Previous Application: {prev.shape}")


✅ Datasets loaded successfully:
Application: (307511, 122)
Bureau: (1716428, 17)
Previous Application: (1670214, 37)


In [34]:
# ===========================================
# 🧾 CELL 3 — Missing Value Summary Function
# ===========================================

def missing_summary(df, name, show_top=20):
    missing = df.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    percent = (missing / len(df)) * 100
    summary = pd.DataFrame({'Missing Values': missing, '% of Total': percent})
    print(f"\n🧾 Missing Summary for {name}: (Top {show_top})")
    display(summary.head(show_top))
    return summary

# Display summaries
missing_summary(app, "Application Data")
missing_summary(bureau, "Bureau Data")
missing_summary(prev, "Previous Application Data")



🧾 Missing Summary for Application Data: (Top 20)


,Missing Values,% of Total
COMMONAREA_MEDI,214865,69.87
COMMONAREA_MODE,214865,69.87
COMMONAREA_AVG,214865,69.87
NONLIVINGAPARTMENTS_MODE,213514,69.43
NONLIVINGAPARTMENTS_MEDI,213514,69.43
NONLIVINGAPARTMENTS_AVG,213514,69.43
FONDKAPREMONT_MODE,210295,68.39
LIVINGAPARTMENTS_AVG,210199,68.35
LIVINGAPARTMENTS_MEDI,210199,68.35
LIVINGAPARTMENTS_MODE,210199,68.35



🧾 Missing Summary for Bureau Data: (Top 20)


,Missing Values,% of Total
AMT_ANNUITY,1226791,71.47
AMT_CREDIT_MAX_OVERDUE,1124488,65.51
DAYS_ENDDATE_FACT,633653,36.92
AMT_CREDIT_SUM_LIMIT,591780,34.48
AMT_CREDIT_SUM_DEBT,257669,15.01
DAYS_CREDIT_ENDDATE,105553,6.15
AMT_CREDIT_SUM,13,0.00



🧾 Missing Summary for Previous Application Data: (Top 20)


,Missing Values,% of Total
RATE_INTEREST_PRIVILEGED,1664263,99.64
RATE_INTEREST_PRIMARY,1664263,99.64
AMT_DOWN_PAYMENT,895844,53.64
RATE_DOWN_PAYMENT,895844,53.64
NAME_TYPE_SUITE,820405,49.12
DAYS_LAST_DUE,673065,40.30
DAYS_FIRST_DRAWING,673065,40.30
DAYS_FIRST_DUE,673065,40.30
DAYS_TERMINATION,673065,40.30
NFLAG_INSURED_ON_APPROVAL,673065,40.30


,Missing Values,% of Total
RATE_INTEREST_PRIVILEGED,1664263,99.64
RATE_INTEREST_PRIMARY,1664263,99.64
AMT_DOWN_PAYMENT,895844,53.64
RATE_DOWN_PAYMENT,895844,53.64
NAME_TYPE_SUITE,820405,49.12
DAYS_LAST_DUE,673065,40.30
DAYS_FIRST_DRAWING,673065,40.30
DAYS_FIRST_DUE,673065,40.30
DAYS_TERMINATION,673065,40.30
NFLAG_INSURED_ON_APPROVAL,673065,40.30


In [35]:
# ===========================================
# 🧹 CELL 4 — Clean Application Data
# ===========================================

# Drop sparsely populated columns (>65% missing)
drop_app_cols = [
    'COMMONAREA_MEDI','COMMONAREA_MODE','COMMONAREA_AVG',
    'NONLIVINGAPARTMENTS_MODE','NONLIVINGAPARTMENTS_MEDI','NONLIVINGAPARTMENTS_AVG',
    'FONDKAPREMONT_MODE','LIVINGAPARTMENTS_AVG','LIVINGAPARTMENTS_MEDI','LIVINGAPARTMENTS_MODE'
]
drop_app_cols = [c for c in drop_app_cols if c in app.columns]

app = app.drop(columns=drop_app_cols)
print(f"Dropped {len(drop_app_cols)} sparse columns from application data ✅")

# Drop any column with >60% missing (except ID and TARGET)
threshold = 0.6
high_missing = app.columns[app.isnull().mean() > threshold].tolist()
high_missing = [c for c in high_missing if c not in ['SK_ID_CURR','TARGET']]

if high_missing:
    print(f"Dropping {len(high_missing)} additional high-missing columns.")
    app = app.drop(columns=high_missing)

# Impute numeric → median, categorical → 'Unknown'
num_cols = [c for c in app.select_dtypes(include=['int64','float64']).columns if c not in ['SK_ID_CURR','TARGET']]
cat_cols = app.select_dtypes(include=['object']).columns.tolist()

num_imputer = SimpleImputer(strategy='median')
if num_cols:
    app[num_cols] = num_imputer.fit_transform(app[num_cols])

if cat_cols:
    app[cat_cols] = app[cat_cols].fillna('Unknown')

print("Application data cleaned ✅")
missing_summary(app, "Application Data (Cleaned)")


Dropped 10 sparse columns from application data ✅
Dropping 7 additional high-missing columns.
Application data cleaned ✅

🧾 Missing Summary for Application Data (Cleaned): (Top 20)


,Missing Values,% of Total


,Missing Values,% of Total


In [36]:
# ===========================================
# 🏦 CELL 5 — Clean Bureau Data
# ===========================================

# Replace numeric NaNs with 0, categorical NaNs with 'Unknown'
num_cols = bureau.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = bureau.select_dtypes(include=['object']).columns.tolist()

bureau[num_cols] = bureau[num_cols].fillna(0)
bureau[cat_cols] = bureau[cat_cols].fillna('Unknown')

print("Bureau data cleaned ✅")
missing_summary(bureau, "Bureau Data (Cleaned)")


Bureau data cleaned ✅

🧾 Missing Summary for Bureau Data (Cleaned): (Top 20)


,Missing Values,% of Total


,Missing Values,% of Total


In [37]:
# ===========================================
# 📂 CELL 6 — Clean Previous Applications Data
# ===========================================

# Drop columns with almost all missing values
drop_prev_cols = [c for c in ['RATE_INTEREST_PRIVILEGED','RATE_INTEREST_PRIMARY'] if c in prev.columns]
if drop_prev_cols:
    prev = prev.drop(columns=drop_prev_cols)
    print(f"Dropped sparse columns: {drop_prev_cols}")

# Numeric → median, Categorical → 'Unknown'
num_cols = [c for c in prev.select_dtypes(include=['int64','float64']).columns if c not in ['SK_ID_CURR','SK_ID_PREV']]
cat_cols = prev.select_dtypes(include=['object']).columns.tolist()

if num_cols:
    prev[num_cols] = prev[num_cols].fillna(prev[num_cols].median())

if cat_cols:
    prev[cat_cols] = prev[cat_cols].fillna('Unknown')

print("Previous Applications data cleaned ✅")
missing_summary(prev, "Previous Applications (Cleaned)")


Dropped sparse columns: ['RATE_INTEREST_PRIVILEGED', 'RATE_INTEREST_PRIMARY']
Previous Applications data cleaned ✅

🧾 Missing Summary for Previous Applications (Cleaned): (Top 20)


,Missing Values,% of Total


,Missing Values,% of Total


In [38]:
# ===========================================
# 💾 CELL 7 — Save Cleaned Datasets
# ===========================================

app.to_csv(save_path + 'application_train_cleaned.csv', index=False)
bureau.to_csv(save_path + 'bureau_cleaned.csv', index=False)
prev.to_csv(save_path + 'previous_application_cleaned.csv', index=False)

print("✅ All cleaned datasets saved successfully!")


✅ All cleaned datasets saved successfully!


In [39]:
 # ===========================================
# 🧠 CELL 8 — Verification Check
# ===========================================

for name, df in zip(['Application', 'Bureau', 'Previous Application'], [app, bureau, prev]):
    total_missing = df.isnull().sum().sum()
    print(f"{name} Data → Shape: {df.shape} | Total Missing: {total_missing}")


Application Data → Shape: (307511, 105) | Total Missing: 0
Bureau Data → Shape: (1716428, 17) | Total Missing: 0
Previous Application Data → Shape: (1670214, 35) | Total Missing: 0
